### Imports

In [8]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.44 ms (started: 2023-07-10 22:48:15 -05:00)


### Parameters

In [9]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = params["method"]
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t\t", option)

if option:
    
    subgroups_id_op = {}
    for group in groups_id:
        subgroups_id_op[group] = [option]
else:
    subgroups_id_op = subgroups_id
print("Subgroups id op:", subgroups_id_op)



Exp:		 exp1
Method:		 vgae
Dimension:	 3
Groups id:	 ['WT', 'zwf1^', 'pck1^']
Subgroups id:	 {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
Option:		 
Subgroups id op: {'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
time: 3.41 ms (started: 2023-07-10 22:48:15 -05:00)


### Edge embeddings

In [3]:
# get edges embeddings

edge_embeddings_global(exp, method, groups_id, subgroups_id_op)

time: 14.9 ms (started: 2023-06-13 13:57:17 -05:00)


In [72]:
df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, groups_id[0], subgroups_id_op[groups_id[0]][0]), index_col=[0, 1])
df_edge_embeddings.head()

0         1         2
0 1  0.279032  1.974517  0.036078
  2  0.261945  3.289689  0.557086
  3  1.843187  2.303246  0.017578
  4  0.199392  3.910711  0.028566
  5  0.000288  6.111294  0.119179

time: 6.09 s (started: 2023-07-07 18:17:47 -05:00)


### Concat edge embeddings

In [5]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    k = 0
    for subgroup in tqdm(subgroups_id_op[group]):
        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, group, subgroup), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)
        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
        k += 1
    
    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index=True)

' for group in tqdm(groups_id):\n    df_edge_embeddings_concat = pd.DataFrame()\n    k = 0\n    for subgroup in tqdm(subgroups_id_op[group]):\n        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, group, subgroup), index_col=[0, 1])\n        df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)\n        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])\n        k += 1\n    \n    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index=True) '

time: 16.2 ms (started: 2023-06-13 13:57:17 -05:00)


In [6]:
df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, groups_id[0], option), index_col=[0, 1])
df_edge_embeddings_concat.head()

' df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, groups_id[0], option), index_col=[0, 1])\ndf_edge_embeddings_concat.head() '

time: 11.8 ms (started: 2023-06-13 13:57:17 -05:00)


In [7]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

' for group in tqdm(groups_id):\n    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])\n\n    x = df_edge_embeddings_concat.iloc[:, 0]\n    y = df_edge_embeddings_concat.iloc[:, 1]\n    z = df_edge_embeddings_concat.iloc[:, 2]\n\n    # Creating figure\n    fig = plt.figure(figsize=(10, 7))\n    ax = plt.axes(projection="3d")\n\n    # Creating plot\n    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)\n    # plt.title("Dimension: {}".format(dimension))\n\n    # show plot\n    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}.png".format(exp, method, group, option))\n    # plt.show()\n    plt.clf() '

time: 12.5 ms (started: 2023-06-13 13:57:17 -05:00)


### Outliers detection

In [8]:
# Outlier detection (HDBSCAN)

""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

' df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])\n\nX_train = df_edge_embeddings_concat.iloc[:, :-1]\nclusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)\n\nthreshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)\noutliers = np.where(clusterer.outlier_scores_ > threshold)[0]\nprint(len(outliers))\noutliers\n\ninliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)\nprint(len(inliers))\ninliers '

time: 4.36 ms (started: 2023-06-13 13:57:17 -05:00)


In [9]:
# outlier detection (ECOD)

dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

100%|██████████| 3/3 [01:06<00:00, 22.08s/it]

time: 1min 6s (started: 2023-06-13 13:57:17 -05:00)


In [10]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
0 1    0.279032  1.974517  0.036078         0
  2    0.261945  3.289689  0.557086         0
  8    0.844659  0.214240  0.160021         0
  9    0.523148  0.000579  0.062768         0
1 131  0.016571  0.177873  0.711692         0

time: 14.4 ms (started: 2023-06-13 13:58:24 -05:00)


In [11]:
# plot outliers/inliers

for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

' for group in tqdm(groups_id):\n    fig = plt.figure(figsize = (10, 7))\n    ax = plt.axes(projection ="3d")\n\n    df_aux = dict_df_edge_embeddings_concat_outlier[group]\n    print("Total:", len(df_aux))\n    \n    temp = df_aux[df_aux["labels"] == 0]\n    x = temp.iloc[:, 0]\n    y = temp.iloc[:, 1]\n    z = temp.iloc[:, 2]\n    ax.scatter3D(x, y, z, c="red", alpha=0.005)\n    print("Num. of inliers:", len(temp))\n\n    temp = df_aux[df_aux["labels"] == 1]\n    x = temp.iloc[:, 0]\n    y = temp.iloc[:, 1]\n    z = temp.iloc[:, 2]\n    ax.scatter3D(x, y, z, c="gray", alpha=0.005)\n    print("Num. of inliers:", len(temp))\n\n    # show plot\n    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}.png".format(exp, method, group, option))\n    # plt.show()\n    plt.clf() '

time: 4.12 ms (started: 2023-06-13 13:58:24 -05:00)


###  Filter common edges

In [12]:
print(subgroups_id)
print(subgroups_id_op)

{'WT': ['1', '2', '3', '4', '5'], 'zwf1^': ['1', '2', '3'], 'pck1^': ['1', '2']}
{'WT': ['dyn'], 'zwf1^': ['dyn'], 'pck1^': ['dyn']}
time: 13.1 ms (started: 2023-06-13 13:58:24 -05:00)


In [13]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id_op[group]:
        df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup),
                               dtype={"id": "string"})
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []
    
    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[subgroups_id_op[group][row[-1]]]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)

11031391it [04:48, 38252.82it/s]it/s]
15391298it [06:46, 37856.48it/s]3, 296.82s/it]
5445463it [02:22, 38192.33it/s]07, 367.46s/it]
100%|██████████| 3/3 [14:21<00:00, 287.06s/it]

time: 14min 21s (started: 2023-06-13 13:58:24 -05:00)


In [14]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
A59.0049  A274.0123  0.279032  1.974517  0.036078         0
          A277.0867  0.261945  3.289689  0.557086         0
          A641.1223  0.844659  0.214240  0.160021         0
          A650.2145  0.523148  0.000579  0.062768         0
A274.0123 A191.046   0.016571  0.177873  0.711692         0

time: 5.56 ms (started: 2023-06-13 14:12:45 -05:00)


In [15]:
# format id
if option:
    for group in tqdm(groups_id):
        # format
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        list_index = []

        for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
            list_index.append((row[0][0][1:], row[0][1][1:]))
        
        # set new index
        df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
        df_edge_embeddings_concat_filter

  0%|          | 0/3 [00:00<?, ?it/s]

11031391it [00:10, 1083651.60it/s]
15391298it [00:14, 1093435.25it/s] 19.71s/it]
5445463it [00:04, 1243280.86it/s], 24.81s/it]
100%|██████████| 3/3 [00:57<00:00, 19.19s/it]

time: 57.6 s (started: 2023-06-13 14:12:45 -05:00)


In [16]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
59.0049  274.0123  0.279032  1.974517  0.036078         0
         277.0867  0.261945  3.289689  0.557086         0
         641.1223  0.844659  0.214240  0.160021         0
         650.2145  0.523148  0.000579  0.062768         0
274.0123 191.046   0.016571  0.177873  0.711692         0

time: 11.7 ms (started: 2023-06-13 14:13:42 -05:00)


In [17]:
# filter diferente edges

if option:
    for group in tqdm(groups_id):
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter.index.get_level_values(0) != df_edge_embeddings_concat_filter.index.get_level_values(1)]
        dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:03<00:00,  1.09s/it]

time: 3.29 s (started: 2023-06-13 14:13:42 -05:00)


In [18]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
print(df_edge_embeddings_concat_filter.shape)
df_edge_embeddings_concat_filter.head()

(11008931, 4)


0         1         2  subgroup
59.0049  274.0123  0.279032  1.974517  0.036078         0
         277.0867  0.261945  3.289689  0.557086         0
         641.1223  0.844659  0.214240  0.160021         0
         650.2145  0.523148  0.000579  0.062768         0
274.0123 191.046   0.016571  0.177873  0.711692         0

time: 12.4 ms (started: 2023-06-13 14:13:46 -05:00)


In [19]:
# count edges and filter by count

dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # read
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    # format
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.rename(columns={"level_0": "source", "level_1": "target"}, inplace=True)
    df_edge_embeddings_concat_filter[["source", "target"]] = df_edge_embeddings_concat_filter[["source", "target"]].astype("string")
    
    # sort edges
    sort_df_edges(df_edge_embeddings_concat_filter)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["source", "target"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]
    
    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:52<00:00, 17.52s/it]

time: 52.6 s (started: 2023-06-13 14:13:46 -05:00)


In [20]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,178.9606,244.9305
1,732.5277,783.4969
2,219.9372,299.8906
3,545.1985,686.1482
4,174.0773,342.1129


time: 7.46 ms (started: 2023-06-13 14:14:38 -05:00)


In [21]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83162 entries, 0 to 83161
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  83162 non-null  object
 1   target  83162 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB
time: 48.8 ms (started: 2023-06-13 14:14:38 -05:00)


In [22]:
# change data type
for group in tqdm(groups_id):
    df_edges_filter = dict_df_edges_filter[group]
    df_edges_filter[["source", "target"]] = df_edges_filter[["source", "target"]].astype("string")

100%|██████████| 3/3 [00:00<00:00, 23.52it/s]

time: 133 ms (started: 2023-06-13 14:14:38 -05:00)


In [23]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,178.9606,244.9305
1,732.5277,783.4969
2,219.9372,299.8906
3,545.1985,686.1482
4,174.0773,342.1129


time: 4.74 ms (started: 2023-06-13 14:14:39 -05:00)


In [24]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83162 entries, 0 to 83161
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   source  83162 non-null  string
 1   target  83162 non-null  string
dtypes: string(2)
memory usage: 1.9 MB
time: 29.5 ms (started: 2023-06-13 14:14:39 -05:00)


In [25]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

100%|██████████| 3/3 [01:09<00:00, 23.32s/it]


,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
67712,100.9679,176.975,0.682490,0.504283,0.548594,0.795642,0.864287
81234,100.9679,260.9136,0.602921,0.526767,0.654005,0.721963,0.852265
60680,102.0562,131.0824,0.828750,0.603544,0.723530,0.860630,0.913332
60723,102.0562,133.014,0.791588,0.641601,0.713314,0.815519,0.875435
60206,102.0562,138.9644,0.596300,0.600574,0.601181,0.807769,0.873217


time: 1min 10s (started: 2023-06-13 14:14:39 -05:00)


In [26]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
67712,100.9679,176.975,0.682490,0.504283,0.548594,0.795642,0.864287
81234,100.9679,260.9136,0.602921,0.526767,0.654005,0.721963,0.852265
60680,102.0562,131.0824,0.828750,0.603544,0.723530,0.860630,0.913332
60723,102.0562,133.014,0.791588,0.641601,0.713314,0.815519,0.875435
60206,102.0562,138.9644,0.596300,0.600574,0.601181,0.807769,0.873217


time: 5.88 ms (started: 2023-06-13 14:15:49 -05:00)


### Filter by STD and average weight

In [27]:
dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:17<00:00,  5.75s/it]


,source,target,weight
0,100.9679,176.975,0.679059
1,100.9679,260.9136,0.671584
2,102.0562,131.0824,0.785957
3,102.0562,133.014,0.767492
4,102.0562,138.9644,0.695808


<Figure size 640x480 with 0 Axes>

time: 17.3 s (started: 2023-06-13 14:15:49 -05:00)


In [28]:
df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, groups_id[0], option),
                              dtype={"source": "string", "target": "string"})
df_common_edges.head()

,source,target,weight
0,100.9679,176.975,0.679059
1,100.9679,260.9136,0.671584
2,102.0562,131.0824,0.785957
3,102.0562,133.014,0.767492
4,102.0562,138.9644,0.695808


time: 43.6 ms (started: 2023-06-13 14:16:06 -05:00)


In [29]:
# show details

for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, option))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

 33%|███▎      | 1/3 [00:00<00:00,  6.60it/s]

Group: WT
Num. nodes: 3618
Num. edges: 81013



 67%|██████▋   | 2/3 [00:01<00:00,  1.12it/s]

Group: zwf1^
Num. nodes: 5423
Num. edges: 740216



100%|██████████| 3/3 [00:02<00:00,  1.12it/s]

Group: pck1^
Num. nodes: 5587
Num. edges: 577311

time: 2.69 s (started: 2023-06-13 14:16:06 -05:00)
